In [1]:
import torch as tr

In [2]:
a = tr.tensor([-10, 10, 10])
tr.exp(a) / tr.exp(a).sum()

tensor([1.0306e-09, 5.0000e-01, 5.0000e-01])

sentences = [
  "How are you",
  "Who are you",
  "Who are they",
  "Who are we",
  "Who am I",
  "Who am I",
  "Where are you going"
]

# replace to training dataset from training datas
## 1. all the elements in sentences are integers, elements equal to words in this model

## 2. read all elements from training data with randomized segments having same orders



In [3]:
path1 = "../data/first_training_set.csv"
path2 = "../data/second_training_set.csv"
path3 = "../data/third_training_set.csv"
path4 = "../data/fourth_training_set.csv"
path5 = "../data/fifth_training_set.csv"


# pick 5000 (each 1000, imply by the parameter inside) segments in each dataset with randomized segments

sentences = []

import number_helpers

sen1 = number_helpers.augment_sentences(path1, 10, 100)
sen2 = number_helpers.augment_sentences(path2, 10, 100)
sen3 = number_helpers.augment_sentences(path3, 10, 100)
sen4 = number_helpers.augment_sentences(path4, 10, 100)
sen5 = number_helpers.augment_sentences(path5, 10, 100)

sentences.extend(sen1)
sentences.extend(sen2)
sentences.extend(sen3)
sentences.extend(sen4)
sentences.extend(sen5)

print(sentences[:5])
print('------------------------------------------------------------------------------------------------------------------------')
print('length:  '+str(len(sentences)))



[['0', '4', '3', '3', '0', '0', '1', '3', '1', '3', '4', '1', '0', '8', '8', '3', '0', '2', '0', '0', '0', '2', '1', '0', '0', '6', '4', '8', '1', '3', '4', '0', '1', '4', '1', '2', '3', '3', '0', '7', '4', '3', '0', '5', '0', '5', '1', '1', '2', '1', '1', '0', '1', '0', '3', '1', '4', '0', '3', '1', '0', '5', '1', '2', '3', '0', '3', '1', '1', '3', '3', '2', '2', '0', '2', '6', '3', '1', '1', '0', '4', '2', '0', '4', '3', '2', '1', '3', '2', '2', '1', '0', '1', '2', '1', '3', '0', '7', '0', '0', '-1'], ['5', '3', '1', '1', '2', '1', '2', '4', '0', '0', '4', '0', '1', '3', '1', '0', '1', '3', '2', '4', '0', '0', '1', '5', '0', '2', '0', '3', '6', '4', '4', '2', '0', '1', '1', '1', '1', '2', '2', '2', '0', '0', '3', '4', '3', '9', '7', '5', '3', '2', '2', '2', '5', '1', '5', '2', '0', '2', '2', '8', '0', '2', '1', '0', '5', '1', '0', '9', '4', '0', '0', '1', '0', '7', '1', '1', '3', '0', '2', '6', '3', '1', '3', '0', '1', '0', '5', '0', '1', '2', '1', '3', '2', '1', '4', '6', '1', '3', 

# start to set building

In [4]:

# Make a dictionary mapping each word to a one-hot tensor
words = set()
for sentence in sentences:
    for word in sentence:
        #for word in sentence.split(" "):
        words.add(word)
words = tuple(words)  # deterministic order

print(words[:5])

# PyTorch LSTM expects 3d tensors representing (sequence length, batch size, number of features)
I = tr.eye(len(words))
dictionary = {
    word: I[w].reshape(1, 1, len(words))
    for w, word in enumerate(words)}

print(len(dictionary))

('6', '5', '7', '0', '9')
11


# try to train the LSTM

In [5]:
# Define a small LSTM recurrent neural network with linear hidden-to-output layer
class Net(tr.nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.lstm = tr.nn.LSTM(input_size=len(words), hidden_size=hidden_size)
        self.readout = tr.nn.Linear(in_features=hidden_size, out_features=len(words))

    def forward(self, x, v=None):
        _, v = self.lstm(x) if v is None else self.lstm(x, v)  # update hidden from input
        h, c = v  # LSTM hidden vector and internal so-called "cell state"
        y = self.readout(h)  # get output from hidden
        y = tr.softmax(y, dim=-1)  # make sure output is a probability distribution
        return y, v


print(Net(3))

Net(
  (lstm): LSTM(11, 3)
  (readout): Linear(in_features=3, out_features=11, bias=True)
)


# next

In [ ]:
net = Net(3)
# opt = tr.optim.SGD(net.parameters(), lr=0.001)
opt = tr.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(200):

    batch_loss = 0.

    for sentence in sentences:
        #tokens = sentence.split(" ")
        tokens = sentence
        v = None  # no hidden activation at first time-step
        for t in range(len(tokens) - 1):
            y, v = net(dictionary[tokens[t]], v)
            y_target = dictionary[tokens[t + 1]]

            #loss = tr.sum((y - y_target)**2) # MSE
            loss = -tr.sum(y_target * tr.log(y))  # Cross-entropy
            batch_loss += loss

    batch_loss.backward()
    opt.step()
    opt.zero_grad()

    print(epoch, batch_loss.item())
    # if epoch % 100 == 0: print(epoch, batch_loss.item())

0 11783.431640625
1 11167.8095703125
2 10845.189453125
3 10580.8095703125
4 10344.470703125
5 10140.90234375
6 9972.6318359375
7 9841.626953125
8 9749.109375
9 9694.251953125
10 9672.443359375
11 9673.556640625
12 9684.4248046875
13 9694.8212890625
14 9699.79296875
15 9697.630859375
16 9689.8232421875
17 9679.7734375
18 9669.59375
19 9659.6806640625
20 9649.4052734375
21 9638.5517578125
22 9627.5751953125
23 9617.4501953125
24 9609.1298828125
25 9602.9833984375
26 9598.9638671875
27 9596.6181640625
28 9595.4765625
29 9595.1533203125
30 9595.296875
31 9595.576171875
32 9595.775390625
33 9595.6220703125
34 9595.0458984375
35 9594.19140625
36 9593.1875
37 9592.267578125
38 9591.4443359375
39 9590.7158203125
40 9590.05078125
41 9589.2666015625
42 9588.447265625
43 9587.548828125
44 9586.7373046875


# complete epoch
### and try prediction

In [8]:
# Try predicting
word = "3"
v = None
print(word)

for t in range(3):
  x = dictionary[word]
  y, v = net(dictionary[tokens[t]], v)
  y = y.squeeze() # ignore singleton dimensions for time-step/example
  w = y.argmax()
  word = words[w]
  prob = y[w]
  print(word, prob.item())

3
0 0.23600423336029053
1 0.23719550669193268
0 0.23819178342819214
